In [ ]:
!pip install pymongo
!pip install praw

In [40]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime

# Conecta-se ao MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['forum_database']  # Banco de dados que as coletas serão armazenadas
posts_collection = db['posts']  # Coleção dos posts coletados

# Função para fazer o scraping e pegar os dados do site
def coletar_dados(url):
    """
    Coleta os dados do site e retorna as informações. Como será utilizado o site Stack Overflow, a estrutura de um post é:
    Título do post
    Conteúdo/Dúvida
    Respostas
    Comentários
    """
    # Faz a requisição à página
    response = requests.get(url)
    
    # Se a requisição for bem-sucedida, procede com o scraping
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Coleta as informações do post
        titulo_post = soup.find('a', class_='question-hyperlink').text.strip()
        
        # Altera para encontrar o conteúdo corretamente
        conteudo_post = soup.find('div', class_='post-text')
        if conteudo_post:
            conteudo_post = conteudo_post.text.strip()
        else:
            conteudo_post = "Conteúdo não disponível"
        
        usuario = soup.find('div', class_='user-details').a.text.strip()
        
        # Tentando encontrar a data do post original
        data_post_tag = soup.find('span', class_='relativetime')
        if data_post_tag:
            data_post_str = data_post_tag['title']  # A data está no atributo 'title'
            data_post = datetime.strptime(data_post_str, '%Y-%m-%d %H:%M:%SZ')  # Novo formato ISO 8601
        else:
            data_post = datetime.now()  # Se não encontrar, utiliza a hora atual
        
        url_post = url  # URL do post

        # Coletando as respostas
        respostas = []
        resposta_divs = soup.find_all('div', class_='answer')
        
        for resposta in resposta_divs:
            # Coleta o autor da resposta
            autor_resposta = resposta.find('div', class_='user-details').a.text.strip()
            
            # Alterando para pegar o conteúdo da resposta com a classe 's-prose'
            conteudo_resposta = resposta.find('div', class_='s-prose')
            if conteudo_resposta:
                conteudo_resposta = conteudo_resposta.text.strip()
            else:
                conteudo_resposta = "Conteúdo da resposta não disponível"
            
            # Coletando a data da resposta
            data_resposta_tag = resposta.find('span', class_='relativetime')
            if data_resposta_tag:
                data_resposta_str = data_resposta_tag['title']
                data_resposta = datetime.strptime(data_resposta_str, '%Y-%m-%d %H:%M:%SZ')
            else:
                data_resposta = datetime.now()  # Se não encontrar, utiliza a hora atual

            # Coleta os comentários da resposta
            comentarios_resposta = []
            comentario_divs = resposta.find_all('div', class_='comment')
            
            for comentario in comentario_divs:
                autor_comentario = comentario.find('a', class_='comment-user').text.strip()
                conteudo_comentario = comentario.find('span', class_='comment-copy').text.strip()
                data_comentario = datetime.now()
                quantidade_reacoes = len(comentario.find_all('span', class_='vote-count-post'))  # Contar reações
                
                comentarios_resposta.append({
                    'autor_comentario': autor_comentario,
                    'data_comentario': data_comentario,
                    'conteudo_comentario': conteudo_comentario,
                    'quantidade_reacoes': quantidade_reacoes
                })
            
            respostas.append({
                'autor_resposta': autor_resposta,
                'conteudo_resposta': conteudo_resposta,
                'data_resposta': data_resposta,
                'comentarios': comentarios_resposta
            })

        # Retorna os dados que foram coletados
        return {
            'titulo_post': titulo_post,
            'conteudo_post': conteudo_post,
            'usuario': usuario,
            'data_post': data_post,
            'url_post': url_post,
            'respostas': respostas
        }
    else:
        print(f"Erro ao acessar a página: {response.status_code}")
        return None

# Função que insere os dados no banco de dados
def inserir_post(data_coleta, url_post, data_post, usuario, titulo_post, conteudo_post, respostas):
    post = {
        "data_coleta": data_coleta,
        "url_post": url_post,
        "data_post": data_post,
        "usuario": usuario,
        "titulo_post": titulo_post,
        "conteudo_post": conteudo_post,
        "respostas": respostas
    }
    result = posts_collection.insert_one(post)
    return result.inserted_id

# URL do site que será realizado a coleta de dados
url_post = "https://stackoverflow.com/questions/3701054/test-code-coverage-without-source-code"

# Coleta os dados do site
dados_coletados = coletar_dados(url_post)

# Se os dados foram coletados com sucesso, devem ser inseridos no banco de dados
if dados_coletados:
    data_coleta = datetime.now()  # Hora atual da coleta
    inserir_post(data_coleta, dados_coletados['url_post'], dados_coletados['data_post'],
                 dados_coletados['usuario'], dados_coletados['titulo_post'],
                 dados_coletados['conteudo_post'], dados_coletados['respostas'])
    print("Post inserido com sucesso!")  # Confirmação positiva
else:
    print("Falha na coleta de dados.")  # Confirmação negativa (provavelmente vai dar erro ao invés de chegar aqui)


Post inserido com sucesso!
